In [1]:
import re

def clean_description(description):    
    description = re.sub(r'http\S+', '', description)
    description = re.sub(r'\S*@\S*\s?', '', description)
    description = re.sub(r'#', '', description)
    description = re.sub(r'(?<![^ .,?!;])podcast(?![^ .:,?!;\r\n])', '', description)
    description = re.sub(r'(?<![^ .,?!;])podcasts(?![^ .,:?!;\r\n])', '', description)
    description = re.sub(r'(?<![^ .,?!;])host(?![^ .,?!:;\r\n])', '', description)
    description = re.sub(r'(?<![^ .,?!;])hosts(?![^ .,?!;\r\n])', '', description)
    description = re.sub(r'(?<![^ .,?!;])episode(?![^ .,:?!;\r\n])', '', description)
    description = re.sub(r'(?<![^ .,?!;])episodes(?![^ .:,?!;\r\n])', '', description)
    description = re.sub(r'(?<![^ .,?!;])sponsor(?![^ .,:?!;\r\n])', '', description)
    description = re.sub(r'(?<![^ .,?!;])sponsored(?![^ .:,?!;\r\n])', '', description)
    return description.lower()


In [9]:
import pandas
from rake_nltk import Rake

# get only the necessary columns from the data and convert to a dataframe
podcasts = pandas.read_csv('metadata.tsv', sep='\t', usecols = ['show_uri', 'show_name', 'language', 'show_description', 'rss_link', 'episode_uri', 'episode_name',	'episode_description',	'duration'], nrows=12000)

# only include podcasts in english language
eng_abr = set(("['en']", "['en-au']", "['en-bz'}", "['en-ca']", "['en-ie']", "['en-jm']", "['en-nz']", "['en-za']", "['en-tt']", "['en-gb']", "['en-us']"))

# ensure all values in language column are lowercase
podcasts['language'] = podcasts['language'].str.lower()

# create dataframe of english podcasts
eng_podcasts = podcasts[podcasts['language'].isin(eng_abr)]
eng_podcasts = eng_podcasts.drop(['language'], axis=1)
eng_podcasts = eng_podcasts[eng_podcasts['episode_description'].apply(lambda x: isinstance(x, str))]
eng_podcasts = eng_podcasts[eng_podcasts['show_description'].apply(lambda x: isinstance(x, str))]

In [3]:
# categorize the podcasts and create category dataframe
# open all xml files in show-rss
import os
import xml.etree.ElementTree as ET
import requests
def parseRSS(rss_link):
   success = False
   ns = {'itunes': 'http://www.itunes.com/dtds/podcast-1.0.dtd'}
   try:
      resp = requests.get(rss_link)
   except:
      resp = None
   if resp != None:
      with open('podcast.xml', 'wb') as f:
         f.write(resp.content)
      categories = set()
      try:
         # create element tree object
         tree = ET.parse('podcast.xml')
         # get root element
         root = tree.getroot()
         for child in root.iter():
            for cat in child.findall('itunes:category', ns):
               category = cat.get('text')
               categories.add(category)
         if len(categories) > 0:
            success = True
      except:
         pass
      finally:
         if os.path.exists("podcast.xml"):
            os.remove("podcast.xml")
         if success:
            return categories
   return None


# create a list of categories and the associated podcast
cat_list = list()
count = 0
wrong = 0
all_categories = set()
# iterate through all columns in dataframe
for ind in eng_podcasts.index:
   if count < 10000:
      uri = eng_podcasts['episode_uri'][ind]
      categories = parseRSS(eng_podcasts['rss_link'][ind])
      if categories != None:
         count += 1
         for cat in categories:
            all_categories.add(cat)
            cat_list.append({'episode_uri': uri, 'category': cat})
      else:
         wrong += 1
         print(wrong)
cat_df = pandas.DataFrame.from_records(cat_list)
      

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class

In [12]:
# create subcategories dataframe
# subcat_df = pandas.DataFrame(columns=['episode_uri', 'subcategory', 'is_power'])
print(len(eng_podcasts))
subcat_list = list()
for index in eng_podcasts.index:
    print(index)
    episode_uri = eng_podcasts['episode_uri'][index]
    episode_desc = clean_description(eng_podcasts['episode_description'][index])
    show_desc = clean_description(eng_podcasts['show_description'][index])
    episode_words = set(episode_desc.split(' '))
    show_words = set(show_desc.split(' '))
    descriptions = episode_desc + episode_desc + episode_desc + show_desc

    r = Rake()
    r.extract_keywords_from_text(descriptions)
    key_phrases = r.get_ranked_phrases_with_scores()
    count_subcategories = 0
    cur_score = 0
    cur_subcategories = set()
    while key_phrases:
        cur_phrase = key_phrases.pop(0)
        if count_subcategories >= 10 or (count_subcategories >= 5 and cur_score != cur_phrase[0]):
            break
        cur_score = cur_phrase[0]
        if cur_phrase[1] in cur_subcategories:
            continue
        cur_subcategories.add(cur_phrase[1])
        is_power_subcategory = False
        if cur_phrase[1].lower() in show_desc and cur_phrase[1] in episode_desc:
            is_power_subcategory = True
        new_item = {'episode_uri': episode_uri, 'subcategory': cur_phrase[1], 'is_power': is_power_subcategory}
        subcat_list.append(new_item)
        count_subcategories += 1


    if index > 5:
        break
subcat_df = pandas.DataFrame.from_records(subcat_list)

8
0
1
2
3
4
5
6


In [5]:
# filter the podcast data frame so that we only have the columns we want
podcast_df = eng_podcasts.drop(['rss_link', 'episode_description', 'show_description'], axis=1)
